## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

In [1]:
import pandas as pd
import nltk
import numpy as np
import scipy
from nltk.metrics.distance import edit_distance
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Лабораторная работа 3

In [2]:
pip install line_profiler

Note: you may need to restart the kernel to use updated packages.


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [3]:
def var_a(reviews):
    sm = c = 0
    
    for row in reviews.iterrows():
        row = row[1]
        if row.date.year==2010:
            sm += row.rating
            c += 1
    
    return sm/c

def var_b(reviews):
    new_table = reviews[reviews['date'].apply(lambda rec: rec.year) == 2010]
    mean = 0
    l = len(new_table)
    
    for row in new_table.iterrows():
        cur = row[1].rating
        mean += (cur/l)
        
    return mean

def var_c(reviews):
    ratings = reviews[reviews['date'].apply(lambda rec: rec.year) == 2010]['rating']
    
    return ratings.mean()

recipes = pd.read_csv("recipes_sample.csv", index_col=1)
reviews = pd.read_csv("reviews_sample.csv", index_col=0, parse_dates=['date'])

print('A: ',end='')
%time var_a(reviews)
print('B: ',end='')
%time var_b(reviews)
print('C: ',end='')
%time var_c(reviews)

A: CPU times: total: 1.02 s
Wall time: 10.3 s
B: CPU times: total: 141 ms
Wall time: 1.43 s
C: CPU times: total: 78.1 ms
Wall time: 535 ms


4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [4]:
%load_ext line_profiler

In [5]:
%lprun -f var_a var_a(reviews)

Timer unit: 1e-07 s

Total time: 29.9753 s
File: C:\Users\ivant\AppData\Local\Temp\ipykernel_15264\1157573881.py
Function: var_a at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def var_a(reviews):
     2         1         16.0     16.0      0.0      sm = c = 0
     3                                               
     4    126697  233945771.0   1846.5     78.0      for row in reviews.iterrows():
     5    126696     878622.0      6.9      0.3          row = row[1]
     6    126696   60161058.0    474.8     20.1          if row.date.year==2010:
     7     12094    4689705.0    387.8      1.6              sm += row.rating
     8     12094      78102.0      6.5      0.0              c += 1
     9                                               
    10         1         21.0     21.0      0.0      return sm/c

In [6]:
%lprun -f var_b var_b(reviews)

Timer unit: 1e-07 s

Total time: 2.06409 s
File: C:\Users\ivant\AppData\Local\Temp\ipykernel_15264\1157573881.py
Function: var_b at line 12

Line #      Hits         Time  Per Hit   % Time  Line Contents
    12                                           def var_b(reviews):
    13         1    2347690.0    2e+06     11.4      new_table = reviews[reviews['date'].apply(lambda rec: rec.year) == 2010]
    14         1          4.0      4.0      0.0      mean = 0
    15         1         57.0     57.0      0.0      l = len(new_table)
    16                                               
    17     12095   14525659.0   1201.0     70.4      for row in new_table.iterrows():
    18     12094    3697362.0    305.7     17.9          cur = row[1].rating
    19     12094      70165.0      5.8      0.3          mean += (cur/l)
    20                                                   
    21         1          3.0      3.0      0.0      return mean

In [7]:
%lprun -f var_c var_c(reviews)

Timer unit: 1e-07 s

Total time: 0.369763 s
File: C:\Users\ivant\AppData\Local\Temp\ipykernel_15264\1157573881.py
Function: var_c at line 23

Line #      Hits         Time  Per Hit   % Time  Line Contents
    23                                           def var_c(reviews):
    24         1    3693192.0    4e+06     99.9      ratings = reviews[reviews['date'].apply(lambda rec: rec.year) == 2010]['rating']
    25                                               
    26         1       4435.0   4435.0      0.1      return ratings.mean()

In [8]:
def var_b(reviews):
    new_table = reviews[reviews['date'].apply(lambda rec: rec.year) == 2010].values[:, 3]
    mean = 0
    l = len(new_table)
    
    for row in new_table:
        cur = row
        mean += (cur/l)
        
    return mean


%time var_b(reviews)

CPU times: total: 31.2 ms
Wall time: 266 ms


4.454440218290292

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [9]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [10]:
%time words_count = get_word_reviews_count(reviews)

CPU times: total: 1.16 s
Wall time: 14.5 s


## Неоптимальная часть кода

В представленном коде есть два основных момента, которые приводят к неэффективности:

1. Дублирование итераций по DataFrame: В первой части кода происходит итерация по DataFrame для создания словаря word_reviews, где для каждого слова формируется список рецептов, в которых оно встречается. Затем, во второй части, снова происходит итерация по DataFrame, но уже для подсчета количества рецептов в каждом списке. Это приводит к дублированию прохода по DataFrame, что снижает производительность.

2. Неэффективное использование словарей: Использование словарей word_reviews и word_reviews_count для хранения информации о словах и количестве их появлений не является оптимальным. При каждом доступе к словарю происходит проверка наличия ключа, что может быть достаточно ресурсоемким при больших объемах данных.


## Оптимизация функции

Для оптимизации функции можно использовать следующие подходы:

1. Объединение итераций: Вместо дублирования итераций по DataFrame можно объединить их в одну, используя метод apply с функцией lambda. 

2. Использование Counter: Вместо словарей word_reviews и word_reviews_count можно использовать класс Counter из модуля collections. Он позволяет эффективно хранить информацию о частоте встречаемости элементов и быстро получать количество вхождений для каждого элемента.


## Оптимизированный код

In [14]:
from collections import Counter
def get_word_reviews_count(df):
    all_count = []
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        counte = Counter(words)
        all_count.extend(list(counte.elements()))
    return Counter(all_count)        
print(get_word_reviews_count(reviews))
%time words_count = get_word_reviews_count(reviews)

Counter({'I': 285147, 'the': 266050, 'and': 217849, '': 214145, 'a': 166136, 'to': 131471, 'for': 121224, 'it': 111175, 'of': 109029, 'was': 88781, 'this': 87057, 'in': 61539, 'with': 58374, 'is': 55075, 'so': 46090, 'used': 44849, 'my': 44144, 'but': 42513, 'recipe': 41098, 'This': 39448, 'made': 38087, 'that': 36984, 'as': 35493, 'have': 35190, 'on': 34583, 'make': 32124, 'Thanks': 26556, 'had': 26362, 'The': 25720, 'very': 25681, 'be': 25032, 'just': 24944, 'you': 24475, 'will': 23985, 'out': 23644, 'not': 22907, 'added': 21710, 'some': 20753, 'were': 20716, 'like': 20171, 'them': 19689, 'It': 19417, 'really': 19400, 'great': 19166, 'time': 17502, 'did': 17358, 'easy': 17222, 'use': 16905, 'they': 16863, 'more': 16840, 'would': 16597, 'little': 16319, 'good': 16106, 'are': 15518, 'all': 15499, 'at': 15362, 'one': 15086, 'also': 14466, 'these': 14418, 'My': 14347, 'recipe.': 14337, "didn't": 14046, 'only': 13965, 'loved': 13755, 'up': 13585, 'add': 12987, 'we': 12841, 'about': 12811,


## Ожидаемый прирост производительности

Оптимизированный код должен демонстрировать значительный (как минимум, на один порядок) прирост производительности по сравнению с исходным кодом, особенно при работе с большими наборами данных. Это связано с устранением дублирования итераций и использованием более эффективных структур данных.

In [12]:
def get_word_reviews_count3(df):
    word_reviews = defaultdict()
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            word_reviews[word].append(recipe_id)

    word_reviews_count = {word: len(recipes) for word, recipes in word_reviews.items()}
    return word_reviews_count

%time words_count = get_word_reviews_count3(reviews)

NameError: name 'defaultdict' is not defined

4. Напишите несколько версий функции `MAPE` для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [13]:
prepared_ratings = reviews.loc[:, ('rating', 'recipe_id')][reviews['rating']>0]

def exctract_by_id(ratings, recipe_id=None):
    try:
        if recipe_id is None:
            ratings_for_id = ratings['rating'].values
        else:
            ratings_for_id = reviews[reviews['recipe_id']==recipe_id].values
    except Exception:
        raise ValueError('There is no reviews for this recipe!')
    
    return ratings_for_id
exctract_by_id(prepared_ratings)

array([5, 5, 4, ..., 5, 5, 5], dtype=int64)